#### Quantization 종류
- 기존의 high precision(일반적으로 fp32) Neural network의 weights와 activation을 더 적은 bit(low precision)으로 변환하는 것
- Quantized Matrix Multiplication, Activation, Layer fusion,,,

#### ex) quantized code
- quantized_cod=uint8(((original-min)/(max-min))*255)

- reconstructured=((quantized_code/255.0)*(max-min)+min

#### Quantization approach 구분
- Post Training Quantization(PTQ): 학습 후에 quantization parameter(scale, shift)를 결정
- Quantization Aware Training(QAT):학습 과정에 quantization을 emulate함으로써, quantization으로 발생하는 성능하락을 완화함

#### PTQ 기법 정리
- Dynamic range quantization(weight only quantization): weight만 quantize됨(8bit), inference 시에는 floating-point로 변환되어 수행
- Full integer quantization(weight and activation quantization): weight와 더불어 모델의 입력 데이터, activation(중간 레이어의 output)들 또한 quantize, Post Training Quantization, Quantization-Aware Training이 여기에 속함
- Float16 quantization: fp32의 데이터 타입의 weight를 fp16으로 quantize

참고: Pytorch 구분
- Dynamic range quantization(Dynamic quantization)을 별도의 범주로 구분
- Post Training Quantization은 Static quantization이라 칭함
- Dynamic quantization의 경우, model 수행 시간이 weights를 load하는 것이 실제 matrix multiplication보다 더 오래 걸리는 LSTM, Transformer 기반의 모델에 효과적이라는 언급이 있음

1) Dynamic range quantization(weight only quantization)
- 네트워크의 Weight만 quantize됨(8bit)
- Pros:
  - 별도의 calibration(validation)데이터가 필요하지 않음
  - 모델의 용량 축소(8bit 기준 1/4)
- Cons:
 - 실제 연산은 floating point로 수행됨

#### 2) Full integer quantization(weight and activation quantization)
- Weight와 더불어 모델의 입력 데이터, activation(중간 레이어의 output)들도 quantize됨
- Pros:
  - 모델의 용량 축소(8bit 기준 1/4)
  - 더 적은 메모리 사용량, cache 재사용성 증가
  - 빠른 연산(fixed point 8bit 연산을 지원하는 경우)
- Cons:
 - Activation의 parameter를 결정하기 위하여 calibration 데이터가 필요함
  (주로 training 데이터에서 사용, 약 100개의 데이터)

2) Full integer quantization; TensorRT(NVIDIA) calibration 예시
- Calibration?; 성능 저하를 최소로 하는 threshold 찾기
- Minimize information loss로 관점으로 접근
- 각 네트워크, 각 레이어 마다 activation value의 range, distribution가 다르다(x축: value, y축: normalized histogram counts)


#### 3) Float16 quantization
- Float 32 모델을 float 16모델로 변환
- Pros:
 - 모델의 용량 축소(1/2)
 - 적은 성능 저하
 - GPU 상에서 빠른 연산(대체로 fp32를 상회)
- Cons:
 - CPU 상에서는 fixed point 연산만큼의 속도 향상이 있지는 않음

#### 1.3 Quantization Aware Training(QAT)
- 학습 과정에서 quantization을 emulate하여(fake quantization), inference시에 발생하는 quantization error를 training 시점에 반영가능하도록 함
- 보통은 일반적인 방법으로 학습을 진행하고, finetuning으로 QAT 적용
- 학습 과정에 emulate된 quantization 파라미터를 inference에도 사용
- PTQ 대비 성능 하락 폭이 적음

- 학습 과정
 - 학습 과정 중 quantization을 적용하고, 다시 floating point로 변환함(backprop을 계산하기 위함)
 - In-out에 대한 gradient를 linear로 가정(straing-through estimator)함으로써 네트워크 학습을 수행